In [14]:
import itertools
import os
import tensorflow as tf
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization, Input, Embedding, LSTM, Dense, Concatenate, Attention
from tensorflow.keras.models import Model
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt



In [ ]:
#ai_corpus had pause and extra info generated by chatgpt 
#ai_corpus_cleaned is the cleaned version
dataframe = pd.read_csv('ai_corpus_cleaned.csv') 
dataframe = dataframe.dropna() #still contained few null/empty cells so removed null 

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from sklearn.model_selection import train_test_split



# Split the data (if needed) or simply use the entire dataset
aave_texts = dataframe['AAVE'].tolist()
sae_texts = dataframe['SAE'].tolist()

# Function to calculate BLEU scores
def calculate_overall_bleu(aave_texts, sae_texts):
    scores = []
    for aave, sae in zip(aave_texts, sae_texts):
        # Tokenize texts - assuming they are already tokenized or split by spaces
        reference = [sae.split()]  # BLEU references need to be in a list of lists
        candidate = aave.split()
        score = sentence_bleu(reference, candidate)
        scores.append(score)
    
    # Calculate the average BLEU score
    average_bleu = sum(scores) / len(scores)
    return average_bleu

# Calculate BLEU score
average_bleu = calculate_overall_bleu(aave_texts, sae_texts)
print(f"Average BLEU score: {average_bleu}")


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import pandas as pd

# Assuming you've loaded your dataset into aave_texts and sae_texts lists

# Prepare the data for corpus_bleu
list_of_references = [[sae.split()] for sae in sae_texts]  # list of reference lists
candidates = [aave.split() for aave in aave_texts]

# Calculate the corpus BLEU score
score = corpus_bleu(list_of_references, candidates)
print(f"Corpus BLEU score: {score}")


In [15]:




dataframe = pd.read_csv('ai_corpus_cleaned.csv')
dataframe = dataframe.dropna()

assert 'AAVE' in dataframe.columns and 'SAE' in dataframe.columns

# Preparing the dataset
aave_texts = dataframe['AAVE'].str.lower().tolist()
sae_texts = dataframe['SAE'].str.lower().tolist()

# Split the data into train and test sets
aave_train, aave_test, sae_train, sae_test = train_test_split(
    aave_texts, sae_texts, test_size=0.2, random_state=21)

# Convert the train and test data into TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices({
    'aave': aave_train,
    'sae': sae_train
})
test_dataset = tf.data.Dataset.from_tensor_slices({
    'aave': aave_test,
    'sae': sae_test
})





BUFFER_SIZE = len(aave_train)  # Use the size of the train dataset

# Adjust batch sizes
train_batch_size = 16 
test_batch_size = 4

# Shuffle and batch the train dataset
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(train_batch_size, drop_remainder=True)

# Batch the test dataset
test_dataset = test_dataset.batch(test_batch_size, drop_remainder=True)

# Text Vectorization
aave_vectorization = TextVectorization(output_mode='int', output_sequence_length=30)
sae_vectorization = TextVectorization(output_mode='int', output_sequence_length=30)

aave_texts = train_dataset.map(lambda x: x['aave'])
sae_texts = train_dataset.map(lambda x: x['sae'])

aave_vectorization.adapt(aave_texts)
sae_vectorization.adapt(sae_texts)

aave_vocab_size = len(aave_vectorization.get_vocabulary())
sae_vocab_size = len(sae_vectorization.get_vocabulary())




In [16]:


# Function to transform the dataset
def split_input_target(batch):
    input_text = batch['aave']
    target_text = batch['sae']
    input_data = aave_vectorization(input_text)
    target_data = sae_vectorization(target_text)
    return {'encoder_input': input_data, 'decoder_input': target_data[:, :-1]}, target_data[:, 1:]

# Apply the function to each item in the dataset
train_dataset = train_dataset.map(split_input_target)
test_dataset = test_dataset.map(split_input_target)


# In[10]:


# Define the ranges for each hyperparameter
patience_range = [10] #5, 8, 10]
units_range = [128]#,256]
learning_rate_range = [0.001]#, 0.01]
activation_functions = ['relu']#, 'tanh']
dropout_rates = [0.5]#, 0.5, 0.7]
regularizers = [None]#, 'l2']
optimizers = ['adam']#, 'rmsprop', 'sgd']
embedding_dim_range = [256]#, 256]  # Added embedding dimension range

#uncommented values results the highest bleu score for this model and also was submitted for the ACM paper


In [17]:
# In[11]:


# Create all combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(
    patience_range, units_range, learning_rate_range, activation_functions, dropout_rates, regularizers, optimizers, embedding_dim_range))

from tensorflow.keras.layers import Bidirectional, Concatenate

def build_model(units, activation, dropout, regularizer, learning_rate, optimizer, embedding_dim):
    # Encoder
    encoder_input = Input(shape=(None,), dtype='int64', name='encoder_input')
    encoder_embedding = Embedding(input_dim=aave_vocab_size, output_dim=embedding_dim)(encoder_input)
    encoder_bilstm = Bidirectional(LSTM(units, return_sequences=True, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=regularizer if regularizer == 'l2' else None))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_embedding)
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    encoder_state = [state_h, state_c]
    
    #encoder output will be contextualized word embeddings

    # Decoder
    decoder_input = Input(shape=(None,), dtype='int64', name='decoder_input')
    decoder_embedding = Embedding(input_dim=sae_vocab_size, output_dim=embedding_dim)(decoder_input)
    decoder_lstm = LSTM(units * 2, return_sequences=True, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=regularizer if regularizer == 'l2' else None)  # Double the units because of BiLSTM
    decoder_lstm_output, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_state)

    # Attention Mechanism
    attention_layer = Attention(use_scale=True)
    attention_output = attention_layer([decoder_lstm_output, encoder_outputs])

    # Concatenation
    decoder_concat_input = Concatenate(axis=-1)([decoder_lstm_output, attention_output])

    # Output Layer
    decoder_dense = Dense(sae_vocab_size, activation='softmax')(decoder_concat_input)

    model = Model([encoder_input, decoder_input], decoder_dense)

    # Compile the model
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

def calculate_bleu_score(model, dataset):
    bleu_scores = []
    smoothie = SmoothingFunction().method4 

    for batch in dataset:
        input_data, target_data = batch
        predictions = np.argmax(model.predict(input_data), axis=-1)

        for input_seq, pred, actual in zip(input_data['encoder_input'], predictions, target_data):
            input_sentence = [aave_vectorization.get_vocabulary()[i] for i in input_seq.numpy() if i != 0]
            pred_sentence = [sae_vectorization.get_vocabulary()[i] for i in pred if i != 0]
            actual_sentence = [sae_vectorization.get_vocabulary()[i] for i in actual.numpy() if i != 0]

            if len(pred_sentence) == 0 or len(actual_sentence) == 0:
                continue

            bleu_score = sentence_bleu([actual_sentence], pred_sentence, smoothing_function=smoothie)
            bleu_scores.append(bleu_score)

    average_bleu_score = np.mean(bleu_scores) if len(bleu_scores) > 0 else 0.0
    return average_bleu_score



def train_and_evaluate_model(patience, units, learning_rate, activation, dropout, regularizer, optimizer,embedding_dim):
    # Build the model with the specified hyperparameters
    model = build_model(units, activation, dropout, regularizer, learning_rate, optimizer,embedding_dim)

    # Early Stopping Callback
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=2)

    # Train the model with a fixed number of epochs (100) but use early stopping
    history = model.fit(train_dataset, epochs=100, validation_data=test_dataset, callbacks=[early_stopping_callback],verbose=2)

    # The epoch at which training stopped
    stopped_epoch = early_stopping_callback.stopped_epoch

    # Evaluate the model (e.g., calculate BLEU score)
    bleu_score = calculate_bleu_score(model, test_dataset)
    
    return bleu_score, model, stopped_epoch, history





In [7]:

results = []

# Iterate over all combinations
for combination in hyperparameter_combinations:
    patience, units, learning_rate, activation, dropout, regularizer, optimizer, embedding_dim = combination
    bleu_score, trained_model, stopped_epoch, history = train_and_evaluate_model(patience, units, learning_rate, activation, dropout, regularizer, optimizer, embedding_dim)

    # Print the results of the current experiment
    print(f"Experiment with patience={patience}, units={units}, learning_rate={learning_rate}, "
          f"activation={activation}, dropout={dropout}, regularizer={regularizer}, optimizer={optimizer}, "
          f"embedding_dim={embedding_dim} completed.")
    print(f"Stopped epoch: {stopped_epoch}, BLEU score: {bleu_score:.4f}\n")
    
    # Save the model
    model_dir_name = f'modelM3_patience{patience}_units{units}_lr{learning_rate}_act{activation}_dropout{dropout}_reg{regularizer}_opt{optimizer}_embed{embedding_dim}'
    os.makedirs(model_dir_name, exist_ok=True)
    model_path = os.path.join(model_dir_name, 'model.h5')
    trained_model.save(model_path)
    
    # Save the loss curve
    plt.figure()
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    loss_curve_path = os.path.join(model_dir_name, 'loss_curve.png')
    plt.savefig(loss_curve_path)
    plt.close()
    
    # Store results
    results.append({
        'patience': patience, 'units': units, 'learning_rate': learning_rate, 'activation': activation, 
        'dropout': dropout, 'regularizer': regularizer, 'optimizer': optimizer, 'embedding_dim': embedding_dim, 'stopped_epoch': stopped_epoch,
        'bleu_score': bleu_score
    })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('hyperparameter_tuning_results_M3.csv', index=False)

print("Hyperparameter tuning completed. Results saved to 'hyperparameter_tuning_results_M3.csv'.")







2024-02-24 10:50:07.850007: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/100


2024-02-24 10:50:13.266817: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f632c003240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-24 10:50:13.266841: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-24 10:50:13.266848: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2024-02-24 10:50:13.271099: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-24 10:50:13.741726: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1708789813.812507 1839663 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1543/1543 - 215s - loss: 2.1315 - accuracy: 0.6784 - val_loss: 1.8606 - val_accuracy: 0.6970 - 215s/epoch - 139ms/step
Epoch 2/100
1543/1543 - 199s - loss: 1.6480 - accuracy: 0.7273 - val_loss: 1.4295 - val_accuracy: 0.7728 - 199s/epoch - 129ms/step
Epoch 3/100
1543/1543 - 198s - loss: 1.1973 - accuracy: 0.8018 - val_loss: 1.1072 - val_accuracy: 0.8285 - 198s/epoch - 129ms/step
Epoch 4/100
1543/1543 - 198s - loss: 0.9317 - accuracy: 0.8380 - val_loss: 1.0071 - val_accuracy: 0.8451 - 198s/epoch - 128ms/step
Epoch 5/100
1543/1543 - 199s - loss: 0.7819 - accuracy: 0.8557 - val_loss: 0.9577 - val_accuracy: 0.8534 - 199s/epoch - 129ms/step
Epoch 6/100
1543/1543 - 198s - loss: 0.6795 - accuracy: 0.8684 - val_loss: 0.9454 - val_accuracy: 0.8562 - 198s/epoch - 128ms/step
Epoch 7/100
1543/1543 - 198s - loss: 0.6038 - accuracy: 0.8797 - val_loss: 0.9447 - val_accuracy: 0.8580 - 198s/epoch - 129ms/step
Epoch 8/100
1543/1543 - 198s - loss: 0.5475 - accuracy: 0.8880 - val_loss: 0.9512 - val_accurac

/home/jupyter-saswar/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Hyperparameter tuning completed. Results saved to 'hyperparameter_tuning_results_M3.csv'.


In [9]:
trained_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            2835968   ['encoder_input[0][0]']       
                                                                                                  
 decoder_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, None, 256),          394240    ['embedding[0][0]']           
 al)                          (None, 128),                                                    

In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import os

# Load the model
model_path = '/home/jupyter-saswar/DL/Untitled Folder/modelM3_patience10_units128_lr0.001_actrelu_dropout0.5_regNone_optadam_embed256/model.h5'  # Update this path
model = tf.keras.models.load_model(model_path)


2024-02-25 17:42:20.086803: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [21]:

# Load your new dataset
new_dataset_path = 'pro_corpus.csv'  # Update this path
new_dataframe = pd.read_csv(new_dataset_path)
new_dataframe = new_dataframe.dropna()

# Assuming your new CSV has the same structure
new_aave_texts = new_dataframe['AAVE'].str.lower().tolist()
new_sae_texts = new_dataframe['SAE'].str.lower().tolist()

# Convert the texts into TensorFlow dataset
# No need to split if you are evaluating the whole new dataset
new_dataset = tf.data.Dataset.from_tensor_slices({
    'aave': new_aave_texts,
    'sae': new_sae_texts
}).batch(4, drop_remainder=True)  


In [11]:
# Function to transform the dataset (assuming aave_vectorization and sae_vectorization are available)
def split_input_target(batch):
    input_text = batch['aave']
    target_text = batch['sae']
    input_data = aave_vectorization(input_text)  # Assuming aave_vectorization is loaded or defined
    target_data = sae_vectorization(target_text)  # Assuming sae_vectorization is loaded or defined
    return {'encoder_input': input_data, 'decoder_input': target_data[:, :-1]}, target_data[:, 1:]

new_dataset = new_dataset.map(split_input_target)

# Calculate the BLEU score on the new dataset
bleu_score = calculate_bleu_score(model, new_dataset)
print(f"BLEU score on the new dataset: {bleu_score:.4f}")

1/1 [==============================] - 0s 29ms/step
BLEU score on the new dataset: 0.2227


In [12]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

 #Split the data into training and testing subsets (80% train, 20% test)
aave_train, aave_test, sae_train, sae_test = train_test_split(
    new_aave_texts, new_sae_texts, test_size=0.2, random_state=21)

# Convert the training and testing data into TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices({
    'aave': aave_train,
    'sae': sae_train
}).batch(16, drop_remainder=True)  # Adjust batch size as needed

test_dataset = tf.data.Dataset.from_tensor_slices({
    'aave': aave_test,
    'sae': sae_test
}).batch(4, drop_remainder=True)  # Adjust batch size as needed

# Assuming aave_vectorization and sae_vectorization are available
train_dataset = train_dataset.map(split_input_target)
test_dataset = test_dataset.map(split_input_target)

# Step 3: Fine-tune the model with Early Stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=5,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,  # To display messages when the callback takes action
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# You may adjust the number of epochs as needed
history = model.fit(
    train_dataset, 
    epochs=50,  # Adjust this value as needed
    validation_data=test_dataset,
    callbacks=[early_stopping_callback]
)

# Step 4: Evaluate the fine-tuned model
bleu_score = calculate_bleu_score(model, test_dataset)
print(f"BLEU score after fine-tuning: {bleu_score:.4f}")

Epoch 1/50


2024-02-25 12:38:41.603417: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f56cc0099b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-25 12:38:41.603446: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-25 12:38:41.603455: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2024-02-25 12:38:41.609116: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-25 12:38:42.110546: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1708882722.183671 1876293 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


289/289 [==============================] - 46s 145ms/step - loss: 0.8997 - accuracy: 0.8825 - val_loss: 0.7662 - val_accuracy: 0.8940
Epoch 2/50
289/289 [==============================] - 36s 123ms/step - loss: 0.5876 - accuracy: 0.9059 - val_loss: 0.6830 - val_accuracy: 0.9055
Epoch 3/50
289/289 [==============================] - 36s 124ms/step - loss: 0.4479 - accuracy: 0.9205 - val_loss: 0.6462 - val_accuracy: 0.9107
Epoch 4/50
289/289 [==============================] - 36s 125ms/step - loss: 0.3589 - accuracy: 0.9309 - val_loss: 0.6271 - val_accuracy: 0.9137
Epoch 5/50
289/289 [==============================] - 36s 125ms/step - loss: 0.2938 - accuracy: 0.9403 - val_loss: 0.6177 - val_accuracy: 0.9157
Epoch 6/50
289/289 [==============================] - 36s 124ms/step - loss: 0.2475 - accuracy: 0.9474 - val_loss: 0.6146 - val_accuracy: 0.9165
Epoch 7/50
289/289 [==============================] - 36s 124ms/step - loss: 0.2135 - accuracy: 0.9530 - val_loss: 0.6175 - val_accuracy: 0.9

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            2835968   ['encoder_input[0][0]']       
                                                                                                  
 decoder_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, None, 256),          394240    ['embedding[0][0]']           
 al)                          (None, 128),                                                    

In [29]:
def translate(model, sentence, aave_vectorization, sae_vectorization, max_seq_length=30):
    # Vectorize the input sentence using the AAVE vectorization layer
    input_vectorized = aave_vectorization([sentence.lower()]).numpy()
    
    # Initialize the sequence with the start-of-sequence token
    # Assuming your sae_vectorization layer uses index 1 for the start token
    target_seq = np.array([[1]])
    
    # Container for the generated sequence
    output_sentence = []
    
    # Loop for max_seq_length steps to generate sequence
    for _ in range(max_seq_length):
        # Predict the next word index
        output_tokens = model.predict([input_vectorized, target_seq])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # Break if end-of-sequence token is generated, assuming it's index 2
        if sampled_token_index == 3:
            break
        
        # Update the target sequence (shifted by one timestep)
        output_sentence.append(sampled_token_index)
        target_seq = np.array([[sampled_token_index]])
    
    # Convert the output sentence indices to words
    translated_sentence = ' '.join([sae_vectorization.get_vocabulary()[i] for i in output_sentence])
    
    return translated_sentence


In [30]:

# Example usage
aave_sentence = "It hard to think that he never care that I love it."
translated_sentence = translate(model, aave_sentence, aave_vectorization, sae_vectorization)
print(f"Translated sentence: {translated_sentence}")


1/1 [==============================] - 0s 18ms/step
Translated sentence: it comes to think it comes to think it comes to think it comes to think it comes to think it comes to think it comes to think it comes


In [31]:
def translate(model, sentence, aave_vectorization, sae_vectorization, max_seq_length=30):
    # Vectorize the input sentence using the AAVE vectorization layer
    input_vectorized = aave_vectorization([sentence.lower()]).numpy()
    
    # Initialize the sequence with the start-of-sequence token
    # Adjust the index according to your sae_vectorization layer's start token
    target_seq = np.array([[1]])  # Assuming index 1 is the start token
    
    # Container for the generated sequence indices
    output_sentence_indices = []
    
    # Loop for max_seq_length steps to generate sequence
    for _ in range(max_seq_length):
        # Predict the next word index
        output_tokens = model.predict([input_vectorized, target_seq])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # Break if end-of-sequence token is generated
        # Adjust the index according to your sae_vectorization layer's end token
        if sampled_token_index == 2:  # Assuming index 2 is the end token
            break
        
        # Append the sampled token index to the output sentence indices
        output_sentence_indices.append(sampled_token_index)
        
        # Update the target sequence by appending the sampled token
        target_seq = np.concatenate([target_seq, [[sampled_token_index]]], axis=-1)
    
    # Convert the output sentence indices to words
    translated_sentence = ' '.join([sae_vectorization.get_vocabulary()[i] for i in output_sentence_indices if i < len(sae_vectorization.get_vocabulary())])
    
    return translated_sentence


In [51]:
dataframe[:15045]

,AAVE,SAE
0,And where were you born?,And where were you born?
1,"I was born in Fort Lauderdale, Florida.","I was born in Fort Lauderdale, Florida."
2,And when did you move here?,When did you relocate to this area?
3,I moved here (pause 0.62) two years after bein...,I moved here two years after I was born.
4,So when you were two?,"So, when you were two years old?"
...,...,...
15165,[Because] they will probably be the ones that ...,They will most likely be the ones who will com...
15166,Uh-huh.,Yes.
15167,"That's the first lesson. The second lesson is,...","That's the initial lesson. The next lesson is,..."
15168,"Because at the time I moved in here, I didn't ...","Because when I first moved into this place, I ..."


In [53]:

# Example usage
aave_sentence = 'So when you were two?'#"Okay. How was that experience?"
translated_sentence = translate(model, aave_sentence, aave_vectorization, sae_vectorization)
print(f"Translated sentence: {translated_sentence}")


1/1 [==============================] - 0s 23ms/step
Translated sentence: so when you were two                         
